In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [19]:
# 10.1 Identifying Missing Data
# We will use the orders table from Super Store for these tasks
orders = pd.read_csv('orders.csv')


# A. Start with some exploratory analysis methods to inspect the data

orders.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'product_id', 'sales', 'quantity', 'discount', 'profit', 'postal_code',
       'region_id'],
      dtype='object')

In [31]:
# B. Which column has the most missing data? Sort the columns by sum of null values
#Postal_code

orders.isna().sum().sort_values(ascending=False)


postal_code    807103
region_id         196
order_id            0
order_date          0
ship_date           0
ship_mode           0
customer_id         0
product_id          0
sales               0
quantity            0
discount            0
profit              0
dtype: int64

In [34]:
# C. Looks like postal_code is our biggest problem, along with region_id
# Use a combination of filtering, isnull, and sum to count how many rows are missing both columns

#see above

In [38]:
# D. Let's drop the region_id nulls from the dataframe before proceeding

orders = orders.dropna(subset=['region_id'])

In [40]:
# E. It's the dream scenario! The IT team confirms all missing postal_code values should be 10001.0

orders['postal_code'] = orders['postal_code'].fillna(10001.0)

/var/folders/1p/qb4fmbcx6d77fyp93p8h41v40000gn/T/ipykernel_739/2577786275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['postal_code'] = orders['postal_code'].fillna(10001.0)


In [63]:
# 10.2 Cleaning Our Data
# A. Write a profit_margin function that accepts a row of data, which is a dictionary
#    It should return the result of dividing the profit column by the sales column (i.e. profit/sales)

def get_profit_margin(profit, sales):
    margin_profit = (profit/sales)*100
    return margin_profit


In [47]:
orders.head(1)

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,10001.0,9954.0


In [64]:
# B. Create a new column in the orders dataframe called 'profit_margin' by applying the profit margin function row-by-row

orders['profit_margin'] = orders.apply(lambda x: get_profit_margin(x['profit'], x['sales']), axis=1)


/var/folders/1p/qb4fmbcx6d77fyp93p8h41v40000gn/T/ipykernel_739/1582931715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['profit_margin'] = orders.apply(lambda x: get_profit_margin(x['profit'], x['sales']), axis=1)


In [65]:
orders.head(1)

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id,profit_margin
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,10001.0,9954.0,0.362889


In [70]:
# C. Use the same process to create a new column called margin_category
# If the profit_margin is less than 0, the margin_category should be "unprofitable"
# If the profit_margin is 0, the margin_category should be "break even"
# If the profit_margin is above 0, the margin_category should "profitable"

def get_margin_category(margin):
    if margin < 0:
        return "unprofitable"
    elif margin == 0:
        return "break even"
    else:
        return "profitable"

orders['margin_category'] = orders.apply(lambda x: get_margin_category(x['profit_margin']), axis=1)

orders['margin_category']

/var/folders/1p/qb4fmbcx6d77fyp93p8h41v40000gn/T/ipykernel_739/207643066.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['margin_category'] = orders.apply(lambda x: get_margin_category(x['profit_margin']), axis=1)


0         profitable
1         profitable
2         profitable
3         profitable
4         profitable
             ...    
999986    profitable
999987    profitable
999988    profitable
999989    profitable
999990    profitable
Name: margin_category, Length: 999795, dtype: object

In [73]:
# D. How many of our orders were unprofitable?

orders.groupby('margin_category').count()['order_id']

margin_category
break even      110925
profitable      876333
unprofitable     12537
Name: order_id, dtype: int64

In [12]:
# 10.3 GroupBy Insights
# Segment the following data and explore aggregate values to answer the following questions

In [81]:
# A. Which discount results in the highest mean order quantity?

orders.groupby('discount').mean()[['quantity']].sort_values(by='quantity', ascending=False)


,quantity
discount,
0.63,4.605882
0.65,4.529412
0.67,3.947368
0.23,3.691551
0.25,3.636766
0.45,3.627625
0.22,3.625110
0.17,3.616540
0.43,3.606561


In [82]:
# B. Which product has the highest mean price discount applied?

orders.groupby('product_id').mean()[['discount']].sort_values(by='discount', ascending=False)

,discount
product_id,
OFF-AP-10002899,0.648485
OFF-AP-10004711,0.648333
OFF-AP-10002933,0.648148
OFF-AP-10001197,0.635769
OFF-AP-10002017,0.623214
...,...
OFF-FA-10003816,0.000000
OFF-PA-10000721,0.000000
TEC-AC-10003469,0.000000


In [15]:
# 10.4 Joining DataFrames
# The below example joins our first two dataframes by their shared column, Symbol
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
combined = pd.merge(openprice, wkhigh, how="left", left_on='Symbol', right_on='Symbol')
combined.head()

,Symbol,OpenPrice,52wkHigh
0,AAPL,217.51,233.47
1,DHR,96.54,110.11
2,DAL,51.45,60.79
3,AMZN,1703.34,2050.49


In [16]:
# A. Join the stockname dataframe to our combined result and print the result
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})


In [ ]:
# B. Use the following tables from Super Store
products = pd.read_csv('./datasets/products.csv')
orders = pd.read_csv('./datasets/orders.csv')
returns = pd.read_csv('./datasets/returns.csv')
regions = pd.read_csv('./datasets/regions.csv')

In [ ]:
# i. We want to join the products and orders dataframes. 
#    Explore both dataframes to identify the common column between them
#    Use a left join to combine the tables in a dataframe named orders_with_products


In [ ]:
# ii. Left join the orders_with_products and returns dataframes


In [ ]:
# iii. Finally, add the region data to our combined dataframe


In [ ]:
# iv. Let's use this combined dataframe to determine the salesperson generating the most profit
